In [2]:
# Common imports
import numpy as np
import os
import pandas as pd
import tensorflow as tf

Adding the data

In [3]:
housing = pd.read_csv("housing.csv")
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [4]:
#Drop the missing values
housing.dropna(axis=0, inplace=True)

# Let's also reset the index
housing.reset_index(inplace=True, drop=True)


housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [5]:
#Set the training and test data sets
housing_num = housing.drop("ocean_proximity", axis=1) # drop labels 

#Select the label
housing_target = housing[["ocean_proximity"]]

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

housing_num_std = scaler.fit_transform(housing_num)

In [7]:
housing_num_std

array([[-1.32731375,  1.05171726,  0.98216331, ..., -0.97683327,
         2.34516291,  2.12881864],
       [-1.32232256,  1.04235526, -0.60621017, ...,  1.67037262,
         2.33263161,  1.31362603],
       [-1.33230494,  1.03767426,  1.85576873, ..., -0.84342665,
         1.78293943,  1.25818254],
       ...,
       [-0.82320322,  1.77727236, -0.92388486, ..., -0.17377773,
        -1.14317103, -0.99247676],
       [-0.87311515,  1.77727236, -0.84446619, ..., -0.39350628,
        -1.05513604, -1.05831591],
       [-0.83318561,  1.74918635, -1.00330353, ...,  0.07995643,
        -0.78060586, -1.01759959]])

In [8]:
housing_num_std.shape

(20433, 9)

In [9]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()

housing_labels_ord = ordinal_encoder.fit_transform(housing_target)

housing_labels_ord[:10]

array([[3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.]])

In [10]:
housing_target[:10]

,ocean_proximity
0,NEAR BAY
1,NEAR BAY
2,NEAR BAY
3,NEAR BAY
4,NEAR BAY
5,NEAR BAY
6,NEAR BAY
7,NEAR BAY
8,NEAR BAY
9,NEAR BAY


In [11]:
# Data type is float. It needs to be integer
housing_labels_ord.dtype

dtype('float64')

In [12]:
#Convert to integer

housing_labels_int = housing_labels_ord.astype(int)

housing_labels_int.dtype

dtype('int32')

In [13]:
housing_labels_int.shape

(20433, 1)

# Split data (train/test)

In [14]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(housing_num_std, housing_labels_int, test_size=0.3)

# Multiclass classification using Keras



In [15]:
from tensorflow import keras

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

# fix random seed for reproducibility
np.random.seed(42)

In [17]:
#Define the model: for multi-class

model = Sequential()

model.add(Dense(50, input_dim=9, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='relu'))

#final layer: there has to be 5 nodes with softmax (because we have 5 categories)
model.add(Dense(5, activation='softmax'))


In [18]:
# Compile model

#Optimizer:
sgd = keras.optimizers.SGD(lr=0.05)


#You need to use "categorical_crossentropy" for mutli-class
#but since our target is ordinal, we need to use "sparse_..."
#if it is binary classification, then use binary_crossentropy

model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

C:\Users\tarunvakkalagadda\Anaconda3\envs\myenv1\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [19]:
# Fit the model

model.fit(train_x, train_y, epochs=100, batch_size=100)

Epoch 1/100
144/144 [==============================] - 1s 714us/step - loss: 1.1812 - accuracy: 0.5181
Epoch 2/100
144/144 [==============================] - 0s 841us/step - loss: 0.8544 - accuracy: 0.6571
Epoch 3/100
144/144 [==============================] - 0s 930us/step - loss: 0.6667 - accuracy: 0.7635
Epoch 4/100
144/144 [==============================] - 0s 848us/step - loss: 0.5609 - accuracy: 0.7924
Epoch 5/100
144/144 [==============================] - 0s 874us/step - loss: 0.4813 - accuracy: 0.8003
Epoch 6/100
144/144 [==============================] - ETA: 0s - loss: 0.4099 - accuracy: 0.84 - 0s 786us/step - loss: 0.4106 - accuracy: 0.8412
Epoch 7/100
144/144 [==============================] - 0s 919us/step - loss: 0.3833 - accuracy: 0.8516
Epoch 8/100
144/144 [==============================] - 0s 1ms/step - loss: 0.3744 - accuracy: 0.8530
Epoch 9/100
144/144 [==============================] - 0s 895us/step - loss: 0.3843 - accuracy: 0.8533
Epoch 10/100
144/144 [===========

In [20]:
# evaluate the model

scores = model.evaluate(test_x, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


192/192 [==============================] - 0s 622us/step - loss: 0.3951 - accuracy: 0.8584

accuracy: 85.84%


# MLP

In [31]:
from sklearn.neural_network import MLPClassifier

mlp_clf = MLPClassifier(hidden_layer_sizes=(50,25,10),
                       max_iter=1000)

mlp_clf.fit(train_x, train_y)

C:\Users\tarunvakkalagadda\Anaconda3\envs\myenv1\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


MLPClassifier(hidden_layer_sizes=(50, 25, 10), max_iter=1000)

In [32]:
test_y_pred = mlp_clf.predict(test_x)

print(accuracy_score(test_y, test_y_pred))

0.933768352365416


By using the same number of nodes in each hidden layers in the above models , we can observe that the MLP performs better.